# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2018 Semester 1
-----
## Project 1: What is labelled data worth to Naive Bayes?
-----
###### Student Name(s): Wenqing Xue
###### Python version: 3.6

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the seven functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [2]:
# Import
import numpy as np
import random
from itertools import permutations

In [3]:
# This function should open a data file in csv, and transform it into a usable format 
def preprocess(filename):
    
    file = open(filename, 'r')
    data = []
    
    for line in file.readlines():
        data.append(line.strip().split(','))
        
    file.close()
    return data

In [4]:
# This function should build a supervised NB model
def train_supervised(data):

    # Prior probability
    prior_dict = {}
    # Posterior probability
    poste_dict = {}
    
    # Calculate prior probability
    for line in data:
        res = line[-1]
        if res not in prior_dict:
            prior_dict[res] = 1
        else:
            prior_dict[res] += 1
            
    for k, v in prior_dict.items():
        prior_dict[k] = v / len(data)
    
    # Calculate posterior probability
    for att in range(len(data[0])-1):
        res_dict = {}
        for line in data:
            res = line[-1]
            if res not in res_dict:
                res_dict[res] = {}
            if line[att] in res_dict[line[-1]]:
                res_dict[res][line[att]] += 1
            else:
                res_dict[res][line[att]] = 1
        poste_dict[att] = res_dict

    return prior_dict, poste_dict

In [5]:
# This function should predict the class for a set of instances, based on a trained model 
def predict_supervised(data, prior_dict, poste_dict):
    
    result = []
    
    sum_value = len(data)
    for line in data:
        class_dict = {}
        for k, v in prior_dict.items():
            class_dict[k] = v
            for i in range(len(line)-1):
                att = line[i]
                if att != '?':
                    sum_dict = sum(poste_dict[i][k].values())
                    if att in poste_dict[i][k]:
#                         class_dict[k] *= poste_dict[i][k][line[i]] / sum(poste_dict[i][k].values())
                        class_dict[k] *= poste_dict[i][k][line[i]] / sum_dict
                    else:
                        # Epsilon smoothing
#                         class_dict[k] *= 0.1 / sum(poste_dict[i][k].values())
                        class_dict[k] *= 0.005 / sum_value
        result.append(max(class_dict, key=class_dict.get))
 
    return result

In [6]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate_supervised(data, result):
    
    count = 0
    for i in range(len(data)):
        if data[i][-1] == result[i]:
            count += 1
    
    accuracy = count / len(data)
    print("Accuracy:", accuracy)
    
    return

In [7]:
def supervised():
    
    data = preprocess('mushroom.csv')
    prior_dict, poste_dict = train_supervised(data)
    result = predict_supervised(data, prior_dict, poste_dict)
    evaluate_supervised(data, result)
    
supervised()

Accuracy: 0.9924913835548991


In [8]:
# This function should build an unsupervised NB model 
def train_unsupervised(data):
    
    # Calculate random values
    class_dict = {} 
    for line in data:
        class_dict[line[-1]] = 0
    
    ins_dict = {}
    for i in range(len(data)):
        random_dict = class_dict.copy()
        sum_random = 0
        for k in random_dict.keys():
            random_value = random.random()
            random_dict[k] = random_value
            sum_random += random_value
            
        for k in class_dict.keys():
            random_dict[k] = random_dict[k] / sum_random

        ins_dict[i] = random_dict
    
    # Prior probability
    prior_dict = class_dict.copy()
    # Posterior probability
    poste_dict = {}
    
    # Calculate prior probability
    for k, v in ins_dict.items():
        for ks, vs in v.items():
            prior_dict[ks] += vs
    
    for k, v in prior_dict.items():
        prior_dict[k] = v / len(ins_dict)
    
    # Calculate posterior probability
    for att in range(len(data[0])-1):
        res_dict = {}
        
        for i in range(len(data)):
            for k in class_dict.keys():
                if k not in res_dict:
                    res_dict[k] = {}

                if data[i][att] not in res_dict[k]:
                    res_dict[k][data[i][att]] = ins_dict[i][k]
                else:
                    res_dict[k][data[i][att]] += ins_dict[i][k]
                    
        poste_dict[att] = res_dict

    for att in range(len(data[0])-1):
        for k in class_dict.keys():
            sum_value = sum(poste_dict[att][k].values())
            for ks, vs in poste_dict[att][k].items():
                poste_dict[att][k][ks] = vs / sum_value
    
    total_dict = {}
    for ins in range(len(data)):
        value_dict = {}
        
        for cla in class_dict.keys():
            value_dict[cla] = prior_dict[cla]
            for att in range(len(data[ins])-1):
                value_dict[cla] *= poste_dict[att][cla][data[ins][att]]
        sum_value = sum(value_dict.values())
        for k, v in value_dict.items():
            value_dict[k] = v / sum_value
        total_dict[ins] = value_dict

    return prior_dict, total_dict

In [9]:
# This function should predict the class distribution for a set of instances, based on a trained model
def predict_unsupervised(data, prior_dict, total_dict):
    
    poste_dict = {}
    
    # Calculate prior probability
    for k, v in total_dict.items():
        for ks, vs in v.items():
            prior_dict[ks] += vs
    
    for k, v in prior_dict.items():
        prior_dict[k] = v / len(total_dict)
    
    # Calculate posterior probability
    for att in range(len(data[0])-1):
        res_dict = {}
        
        for i in range(len(data)):
            for k in prior_dict.keys():
                if k not in res_dict:
                    res_dict[k] = {}
                if data[i][att] != '?':
                    if data[i][att] not in res_dict[k]:
                        res_dict[k][data[i][att]] = total_dict[i][k]
                    else:
                        res_dict[k][data[i][att]] += total_dict[i][k]
                    
        poste_dict[att] = res_dict

    for att in range(len(data[0])-1):
        for k in prior_dict.keys():
            sum_value = sum(poste_dict[att][k].values())
            for ks, vs in poste_dict[att][k].items():
                poste_dict[att][k][ks] = vs / sum_value
    
    tmp_dict = {}
    for ins in range(len(data)):
        value_dict = {}
        
        for cla in prior_dict.keys():
            value_dict[cla] = prior_dict[cla]
            for att in range(len(data[ins])-1):
                if data[ins][att] != '?':
                    value_dict[cla] *= poste_dict[att][cla][data[ins][att]]
        sum_value = sum(value_dict.values())
        for k, v in value_dict.items():
            value_dict[k] = v / sum_value
        tmp_dict[ins] = value_dict
    
    return prior_dict, tmp_dict

In [10]:
# This function should evaluate a set of predictions, in an unsupervised manner
def evaluate_unsupervised(data, result):
    
    count = 0
    for i in range(len(data)):
        if data[i][-1] == result[i]:
            count += 1
    
    accuracy = count / len(data)
    print("Accuracy:", accuracy)
    
    return accuracy

In [12]:
def unsupervised():
    
    data = preprocess('mushroom.csv')
    prior_dict, total_dict = train_unsupervised(data)
    
    for i in range(5):
        prior_dict, total_dict = predict_unsupervised(data, prior_dict, total_dict)
        
    result = []
    for i in range(len(data)):
        result.append(max(total_dict[i], key=total_dict[i].get))
    
    prior_list = list(prior_dict)
    classifier = []
    for r in result:
        for i in range(len(prior_list)):
            if r == prior_list[i]:
                classifier.append(i)
    
    class_num = len(set(result))
    class_list = list(set(result))
    class_list *= class_num

    predict_class = set()
    for i in list(permutations(class_list, class_num)):
        predict_class.add(i)
        
    actual_list = [line[-1] for line in data]
    
    result_list = []
    for clas in list(predict_class):
        res = []
        correct = 0
        for i in classifier:
            res.append(clas[i])
        result_list.append(res)
    
    count_list = []
    for res in result_list:
        count = 0
        for i in range(len(res)):
            if res[i] == actual_list[i]:
                count += 1
        count_list.append(count/len(res))

    index = count_list.index(max(count_list))
    swapped_result = result_list[index]
    
    acc = evaluate_unsupervised(data, swapped_result)
    return acc


count = 0
for i in range(10):
    acc = unsupervised()
    count += acc
print("Average:", count / 10)
print('-'*80)

# unsupervised()

Accuracy: 0.8924175283111767
Accuracy: 0.8924175283111767
Accuracy: 0.5999507631708518
Accuracy: 0.8724766125061546
Accuracy: 0.8916789758739537
Accuracy: 0.5226489414081733
Accuracy: 0.8417035942885278
Accuracy: 0.8959871984244214
Accuracy: 0.8645987198424422
Accuracy: 0.8920482520925652
Average: 0.8165928114229443
--------------------------------------------------------------------------------


Questions (you may respond in a cell or cells below):

1. Since we’re starting off with random guesses, it might be surprising that the unsupervised NB works at all. Explain what characteristics of the data cause it to work pretty well (say, within 10% Accuracy of the supervised NB) most of the time; also, explain why it utterly fails sometimes.
2. When evaluating supervised NB across the four different datasets, you will observe some variation in effectiveness (e.g. Accuracy). Explain what causes this variation. Describe and explain any particularly suprising results.
3. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out (hint: check out numpy.shuffle()) or cross–validation evaluation strategy. How does your estimate of Accuracy change, compared to testing on the training data? Explain why. (The result might surprise you!)
4. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Do you notice any variation in the predictions made by either the supervised or unsupervised NB classifiers? Explain why, or why not.
5. The lecture suggests that deterministically labelling the instances in the initialisation phase of the unsupervised NB classifier “doesn’t work very well”. Confirm this for yourself, and then demonstrate why.
6. Rather than evaluating the unsupervised NB classifier by assigning a class deterministically, instead calculate how far away the probabilistic estimate of the true class is from 1 (where we would be certain of the correct class), and take the average over the instances. Does this performance estimate change, as we alter the number of iterations in the method? Explain why.
7. Explore what causes the unsupervised NB classifier to converge: what proportion of instances change their prediction from the random assignment, to the first iteration? From the first to the second? What is the latest iteration where you observe a prediction change? Make some conjecture(s) as to what is occurring here.

Don't forget that groups of 1 student should respond to question (1), and one other question. Groups of 2 students should respond to question (1), and three other questions. Your responses should be about 100-200 words each.

### Question 1 Answer
---


### Question 0 Answer
---
